In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelBinarizer
from sklearn.feature_selection import VarianceThreshold

# Global Parameters

In [2]:
REALIZED_DATA_PATH = 'Realized Schedule 20210101-20220228.xlsx'
FUTURE_DATA_PATH = 'Future Schedule 20220301-20220331.xlsx'

# Data Preprocessing of Case 1
### By August Semrau and William Marstrand
This notebook consists of data preprocessing, analysis and modelling is found in seperate notebooks

In [3]:
### Load data
df_realized = pd.read_excel(REALIZED_DATA_PATH)
df_realized

,ScheduleTime,Airline,FlightNumber,Destination,AircraftType,FlightType,Sector,SeatCapacity,LoadFactor
0,2021-01-01 06:35:00,IA,874,DEN,73W,J,US,142,0.408451
1,2021-01-01 10:35:00,JZ,818,YHM,AT7,J,CA,74,0.189189
2,2021-01-01 12:05:00,IA,876,DEN,73W,J,US,142,0.570423
3,2021-01-01 13:20:00,CN,514,EST,AT7,J,US,72,0.333333
4,2021-01-01 14:20:00,LJ,3140,DEN,32A,J,US,186,0.204301
...,...,...,...,...,...,...,...,...,...
39444,2022-02-28 18:45:00,VW,986,YYZ,319,J,CA,144,0.847222
39445,2022-02-28 19:25:00,LJ,667,YDQ,319,J,CA,156,0.871795
39446,2022-02-28 20:00:00,VW,3406,MYV,E90,J,US,98,0.857143
39447,2022-02-28 19:30:00,LJ,663,RUT,32N,J,US,186,0.682796


In [4]:
df_future = pd.read_excel(FUTURE_DATA_PATH)
df_future

,ScheduleTime,Airline,FlightNumber,Destination,AircraftType,FlightType,Sector,SeatCapacity
0,2022-03-01 05:45:00,UK,1315,YXX,319,J,CA,131
1,2022-03-01 07:15:00,PW,950,YLW,319,J,CA,143
2,2022-03-01 06:45:00,VW,770,YYZ,320,J,CA,220
3,2022-03-01 06:40:00,DO,2568,YXU,32N,J,CA,180
4,2022-03-01 07:30:00,AY,984,YYG,320,J,CA,174
...,...,...,...,...,...,...,...,...
4808,2022-03-31 21:05:00,SV,1988,YYJ,73H,J,CA,186
4809,2022-03-31 20:55:00,MD,1242,YBR,321,J,CA,230
4810,2022-03-31 21:50:00,LJ,506,YUL,320,J,CA,186
4811,2022-03-31 20:30:00,LJ,772,YEG,320,J,CA,186


First we convert the flight number to an object type, since it is really a categorical variable, but Pandas thinks it's numerical

In [5]:
## Convert FlightNumber to object, it is not a numerical value
df_realized.FlightNumber = df_realized.FlightNumber.astype(object)
df_future.FlightNumber = df_realized.FlightNumber.astype(object)
## Investigate data for dtypes and stuff
print('\ndtypes of the datasets columns:')
df_realized.dtypes


dtypes of the datasets columns:


ScheduleTime    datetime64[ns]
Airline                 object
FlightNumber            object
Destination             object
AircraftType            object
FlightType              object
Sector                  object
SeatCapacity             int64
LoadFactor             float64
dtype: object

## Feature Engineering
Based on the different data types, we already know we want to do a couple of things to each column/feature, and these are described below:
<br><br>
For **ScheduleTime**, we want to do two things; Firstly, we want to convert the datetime64[ns] format to a more simple datetime format. We further want to create new object columns; one which encodes which year it is, one for the month of the year the flight is, which week, weekday, hour of day and minute of hour. These will also be encoded in the next step.
These new features extrated from the timestamp will be categorical features.
<br><br>
For **Airline**, **Flightnumber** (infact categorical), **Destination**, **Aircrafttype**, **Flighttype** and **Sector**, columns which are encoded as objects (they are categorical), we need to make an alternative encoding. If they hold few different classes, this is true for Flighttype which is either *J* or *C*, we will one-hot-encode them, else they will be label encoded. 
<br><br>
SeatCapacity is made categorical, though it differently from the rest, is in fact ordinal.
<br><br>
First, we remove rows with empty values

In [6]:
### Investigate for missing data
rea_missing_values_count = df_realized.isnull().sum()
print(f'Realized data, List of null data counts for each column: \n{rea_missing_values_count}')
fut_missing_values_count = df_future.isnull().sum()
print(f'Future data, List of null data counts for each column: \n{fut_missing_values_count}\n')

## As there are so few missing values, a total of five rows, we simply remove these
rea_df = df_realized.dropna()
# rea_df = rea_df.loc[rea_df['FlightType'].isin(['C', 'J'])]
print(f'\nRealized data had {df_realized.shape[0]} rows pre-nan-removal, now has {rea_df.shape[0]} rows.')

fut_df = df_future.dropna()
# fut_df = fut_df.loc[fut_df['FlightType'].isin(['C', 'J'])]
print(f'\nFuture data had {df_future.shape[0]} rows pre-nan-removal, now has {fut_df.shape[0]} rows.\n')

## Seperate data from target
rea_X, rea_y = rea_df.loc[:, rea_df.columns != 'LoadFactor'], rea_df.loc[:, rea_df.columns == 'LoadFactor']
fut_X, fut_y = fut_df.loc[:, fut_df.columns != 'LoadFactor'], fut_df.loc[:, fut_df.columns == 'LoadFactor']

Realized data, List of null data counts for each column: 
ScheduleTime    0
Airline         0
FlightNumber    0
Destination     0
AircraftType    0
FlightType      0
Sector          0
SeatCapacity    0
LoadFactor      0
dtype: int64
Future data, List of null data counts for each column: 
ScheduleTime    0
Airline         0
FlightNumber    0
Destination     0
AircraftType    0
FlightType      0
Sector          0
SeatCapacity    0
dtype: int64


Realized data had 39449 rows pre-nan-removal, now has 39449 rows.

Future data had 4813 rows pre-nan-removal, now has 4813 rows.



## Preprocess **ScheduleTime**

In [7]:
### Preprosses time for getting year, month, week number, weekday, hour of day and minute of hour
rea_X['Year'] = rea_X['ScheduleTime'].dt.year
rea_X['Month'] = rea_X['ScheduleTime'].dt.month
rea_X['WeekNumber'] = rea_X['ScheduleTime'].dt.isocalendar().week % 52;
rea_X['Weekday'] = rea_X['ScheduleTime'].dt.dayofweek;
rea_X['HourOfDay'] = rea_X['ScheduleTime'].dt.hour
rea_X['MinuteOfHour'] = rea_X['ScheduleTime'].dt.minute

rea_X.Year = rea_X.Year.astype(object)
rea_X.Month = rea_X.Month.astype(object)
rea_X.WeekNumber = rea_X.WeekNumber.astype(object)
rea_X.Weekday = rea_X.Weekday.astype(object)
rea_X.HourOfDay = rea_X.HourOfDay.astype(object)
rea_X.MinuteOfHour = rea_X.MinuteOfHour.astype(object)

fut_X['Year'] = fut_X['ScheduleTime'].dt.year
fut_X['Month'] = fut_X['ScheduleTime'].dt.month
fut_X['WeekNumber'] = fut_X['ScheduleTime'].dt.isocalendar().week % 52;
fut_X['Weekday'] = fut_X['ScheduleTime'].dt.dayofweek;
fut_X['HourOfDay'] = fut_X['ScheduleTime'].dt.hour
fut_X['MinuteOfHour'] = fut_X['ScheduleTime'].dt.minute

fut_X.Year = fut_X.Year.astype(object)
fut_X.Month = fut_X.Month.astype(object)
fut_X.WeekNumber = fut_X.WeekNumber.astype(object)
fut_X.Weekday = fut_X.Weekday.astype(object)
fut_X.HourOfDay = fut_X.HourOfDay.astype(object)
fut_X.MinuteOfHour = fut_X.MinuteOfHour.astype(object)

rea_X = rea_X.drop('ScheduleTime', axis=1)
fut_X = fut_X.drop('ScheduleTime', axis=1)

## Preprocess categorical features

In [8]:
### We need to find out how many categorical values exist for each feature
rea_X.describe(include = ['object'])

,Airline,FlightNumber,Destination,AircraftType,FlightType,Sector,Year,Month,WeekNumber,Weekday,HourOfDay,MinuteOfHour
count,39449,39449,39449,39449,39449,39449,39449,39449,39449,39449,39449,39449
unique,105,825,230,58,3,12,2,12,52,7,23,13
top,DO,771,CKE,73H,J,CA,2021,10,1,6,8,40
freq,10483,366,2706,8970,38229,19311,32348,4705,1427,6495,3004,3368


In [9]:
fut_X.describe(include = ['object'])

,Airline,FlightNumber,Destination,AircraftType,FlightType,Sector,Year,Month,WeekNumber,Weekday,HourOfDay,MinuteOfHour
count,4813,4813,4813,4813,4813,4813,4813,4813,4813,4813,4813,4813
unique,71,260,172,38,4,10,1,1,5,7,24,12
top,DO,556,DEN,73H,J,CA,2022,3,12,3,8,40
freq,1256,124,255,1169,4693,2434,4813,4813,1086,791,414,435


In [10]:
nominal_cols = ['Airline', 'FlightNumber', 'Destination', 'AircraftType', 'FlightType', 'Sector', 'Year', 'Month', 'WeekNumber', 'Weekday', 'HourOfDay', 'MinuteOfHour']
for col in nominal_cols:
    print(f'Realized data, For feature {col}, there are {rea_X[rea_X.groupby(by=col).transform(len) == 1].count()[0]} values that only occur once')
    print(f'Future data, For feature {col}, there are {fut_X[fut_X.groupby(by=col).transform(len) == 1].count()[0]} values that only occur once')



Realized data, For feature Airline, there are 0 values that only occur once
Future data, For feature Airline, there are 0 values that only occur once
Realized data, For feature FlightNumber, there are 121 values that only occur once
Future data, For feature FlightNumber, there are 47 values that only occur once
Realized data, For feature Destination, there are 19 values that only occur once
Future data, For feature Destination, there are 16 values that only occur once
Realized data, For feature AircraftType, there are 7 values that only occur once
Future data, For feature AircraftType, there are 1 values that only occur once
Realized data, For feature FlightType, there are 0 values that only occur once
Future data, For feature FlightType, there are 1 values that only occur once
Realized data, For feature Sector, there are 1 values that only occur once
Future data, For feature Sector, there are 0 values that only occur once
Realized data, For feature Year, there are 0 values that only o

### Encoding

In [11]:
nominal_cols = ['AircraftType', 'FlightType', 'Sector', 'Year', 'Month', 'WeekNumber', 'Weekday', 'HourOfDay'] 
linear_rea_X = pd.get_dummies(rea_X, columns=nominal_cols, drop_first=True)
linear_fut_X = pd.get_dummies(fut_X, columns=nominal_cols, drop_first=True)
tree_rea_X = pd.get_dummies(rea_X, columns=nominal_cols, drop_first=False)
tree_fut_X = pd.get_dummies(fut_X, columns=nominal_cols, drop_first=False)

/opt/homebrew/Caskroom/miniconda/base/envs/airport_forecasting/lib/python3.8/site-packages/pandas/core/algorithms.py:798: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  uniques = Index(uniques)
/opt/homebrew/Caskroom/miniconda/base/envs/airport_forecasting/lib/python3.8/site-packages/pandas/core/algorithms.py:798: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  uniques = Index(uniques)
/opt/homebrew/Caskroom/miniconda/base/envs/airport_forecasting/lib/python3.8/site-packages/pandas/core/algorithms.py:798: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  uniques = Index(uniques)
/opt/homebrew/Caskroom/miniconda/base/envs/airport_forecasting/lib/python3.8/site-packages/pandas/core/

In [12]:
high_count_nominal_cols = ['MinuteOfHour','Airline', 'FlightNumber', 'Destination']
lb = LabelBinarizer()

for col in tqdm(high_count_nominal_cols):
    linear_rea_X[col] = lb.fit_transform(linear_rea_X[col].tolist())
    linear_fut_X[col] = lb.fit_transform(linear_fut_X[col].tolist())
    tree_rea_X[col] = lb.fit_transform(tree_rea_X[col].tolist())
    tree_fut_X[col] = lb.fit_transform(tree_fut_X[col].tolist())


100%|██████████| 4/4 [00:00<00:00,  4.11it/s]


In [13]:
linear_fut_X

,Airline,FlightNumber,Destination,SeatCapacity,MinuteOfHour,AircraftType_223,AircraftType_295,AircraftType_319,AircraftType_320,AircraftType_321,...,HourOfDay_14,HourOfDay_15,HourOfDay_16,HourOfDay_17,HourOfDay_18,HourOfDay_19,HourOfDay_20,HourOfDay_21,HourOfDay_22,HourOfDay_23
0,0,0,0,131,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,143,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,220,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,180,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,174,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4808,0,0,0,186,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4809,0,0,0,230,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
4810,0,0,0,186,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
4811,0,0,0,186,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0


In [14]:
linear_rea_X

,Airline,FlightNumber,Destination,SeatCapacity,MinuteOfHour,AircraftType_223,AircraftType_295,AircraftType_318,AircraftType_319,AircraftType_320,...,HourOfDay_14,HourOfDay_15,HourOfDay_16,HourOfDay_17,HourOfDay_18,HourOfDay_19,HourOfDay_20,HourOfDay_21,HourOfDay_22,HourOfDay_23
0,0,0,0,142,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,74,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,142,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,72,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,186,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39444,0,0,0,144,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
39445,0,0,0,156,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
39446,0,0,0,98,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
39447,0,0,0,186,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


Now we remove any columns that holds identical values for all rows i.e. has no information to differentiate data points

In [15]:
selector = VarianceThreshold(0)
linear_rea_X = pd.DataFrame(selector.fit_transform(linear_rea_X), columns=selector.get_feature_names_out())
linear_fut_X = pd.DataFrame(selector.fit_transform(linear_fut_X), columns=selector.get_feature_names_out())

print(linear_rea_X.shape)
print(linear_fut_X.shape)

(39449, 166)
(4813, 87)


In [16]:
tree_rea_X = pd.DataFrame(selector.fit_transform(tree_rea_X), columns=selector.get_feature_names_out())
tree_fut_X = pd.DataFrame(selector.fit_transform(tree_fut_X), columns=selector.get_feature_names_out())

print(tree_rea_X.shape)
print(tree_fut_X.shape)

(39449, 174)
(4813, 93)


We then remove all columns where only a single records has the value

In [17]:
linear_rea_X = linear_rea_X[linear_rea_X.columns[linear_rea_X.sum(axis=0) > 10]]
linear_fut_X = linear_fut_X[linear_fut_X.columns[linear_fut_X.sum(axis=0) > 10]]
tree_rea_X = tree_rea_X[tree_rea_X.columns[tree_rea_X.sum(axis=0) > 10]]
tree_fut_X = tree_fut_X[tree_fut_X.columns[tree_fut_X.sum(axis=0) > 10]]

We now only choose the columns that are both in the future and realized data

In [18]:
rea_fut_cols = list(set(linear_rea_X.columns).intersection(set(linear_fut_X.columns)))

linear_rea_X = linear_rea_X[rea_fut_cols]
linear_fut_X = linear_fut_X[rea_fut_cols]
print(f'Number of cols: {len(rea_fut_cols)}')

Number of cols: 66


In [19]:
tree_rea_fut_cols = list(set(tree_rea_X.columns).intersection(set(tree_fut_X.columns)))

tree_rea_X = tree_rea_X[tree_rea_fut_cols]
tree_fut_X = tree_fut_X[tree_rea_fut_cols]
print(f'Number of cols: {len(tree_rea_fut_cols)}')

Number of cols: 70


### Distribution Similarity
It is important that realized training features and future prediction features have a similar distribution.

In [20]:
features = tree_rea_fut_cols

fig, ax = plt.subplots(len(features), 2, figsize=(15, 500))

for i, feature in enumerate(tqdm(features)):
    ax[i,0].set_title(f'Realized {feature}')
    sns.histplot(tree_rea_X[feature], ax=ax[i,0])
    ax[i,1].set_title(f'Future {feature}')
    sns.histplot(tree_fut_X[feature], ax=ax[i,1])

plt.tight_layout()
plt.show()


100%|██████████| 70/70 [00:03<00:00, 19.87it/s]


KeyboardInterrupt: 